# Use Spark and batch deployment to predict customer churn with `ibm-watson-machine-learning`

This notebook contains steps and code to develop a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook runs on Python and Spark.

You will use a data set, **Telco Customer Churn**, which details anonymous customer data from a telecommunication company. Use the details of this data set to predict customer churn which is very critical to business as it's easier to retain existing customers rather than acquiring new ones.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Explore and visualize prediction results using the plotly package.
-  Deploy a model for batch scoring using Wastson Machine Learning API.


## Contents

This notebook contains the following parts:

1. [Set up the environment](#setup)
2. [Load and explore data](#load)
3. [Create spark ml model](#model)
4. [Persist model](#persistence)
5. [Predict locally and visualize](#visualization)
6. [Deploy and score in a Cloud](#scoring)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/admin/create-services.html?context=cpdaas&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="https://ibm.github.io/watson-machine-learning-sdk/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!rm -rf /home/spark/shared/user-libs/python3.10*

In [ ]:
!pip install -U --user ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First, create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

### Test Spark

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    print('Error: Spark runtime is missing. If you are using Watson Studio change the notebook runtime to Spark.')
    raise

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration. 

In [ ]:
!pip install wget

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'WA_FnUseC_TelcoCustomerChurn.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/customer_churn/WA_FnUseC_TelcoCustomerChurn.csv", out=sample_dir)

In [ ]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .option('nanValue', ' ')\
  .option('nullValue', ' ')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all records
-  show distribution of label classes

In [ ]:
df_data.printSchema()

print("Number of fields: %3g" % len(df_data.schema))

As you can see, the data contains 21 fields. "Churn" field is the one we would like to predict (label).

In [ ]:
print("Total number of records: " + str(df_data.count()))

Data set contains 7043 records.

Now you will check if all records have complete data.

In [ ]:
df_complete = df_data.dropna()

print("Number of records with complete data: %3g" % df_complete.count())

You can see that there are some missing values you can investigate that all missing values are present in `TotalCharges` feature. We will use dataset with missing values removed for model training and evaluation.


Now you will inspect distribution of classes in label column.

In [ ]:
df_complete.groupBy('Churn').count().show()

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [ ]:
(train_data, test_data, predict_data) = df_complete.randomSplit([0.8, 0.18, 0.02], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))
print("Number of records for prediction: " + str(predict_data.count()))

As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the predictors to features vector and label feature convert to numeric.

In [ ]:
lab = StringIndexer(inputCol = 'Churn', outputCol = 'label')
features = RFormula(formula = "~ gender + SeniorCitizen +  Partner + Dependents + tenure + PhoneService + MultipleLines + InternetService + OnlineSecurity + OnlineBackup + DeviceProtection + TechSupport + StreamingTV + StreamingMovies + Contract + PaperlessBilling + PaymentMethod + MonthlyCharges + TotalCharges - 1")

Next, define estimators you want to use for classification. Logistic Regression is used in the following example.

In [ ]:
lr = LogisticRegression(maxIter = 10)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [ ]:
pipeline_lr = Pipeline(stages = [features, lab , lr])

Now, you can train your Logistic Regression model using the previously defined **pipeline** and **train data**."

In [ ]:
model_lr = pipeline_lr.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [ ]:
predictions = model_lr.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Test dataset:")
print("Accuracy = %3.2f" % accuracy)

You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model


In this section you will learn how to store your pipeline and model in Watson Machine Learning repository using Python client libraries.

#### Save training data in your Cloud Object Storage

ibm-cos-sdk library allows Python developers to manage Cloud Object Storage (COS).

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

**Action**: Put credentials from Object Storage Service in Bluemix here.

In [ ]:
cos_credentials = {
                  "apikey": "***",
                  "cos_hmac_keys": {
                    "access_key_id": "***",
                    "secret_access_key": "***"
                  },
                  "endpoints": "***",
                  "iam_apikey_description": "***",
                  "iam_apikey_name": "***",
                  "iam_role_crn": "***",
                  "iam_serviceid_crn": "***",
                  "resource_instance_id": "***"
                }

In [ ]:
connection_apikey = cos_credentials['apikey']
connection_resource_instance_id = cos_credentials["resource_instance_id"]
connection_access_key_id = cos_credentials['cos_hmac_keys']['access_key_id']
connection_secret_access_key = cos_credentials['cos_hmac_keys']['secret_access_key']

**Action**: Define the service endpoint we will use. <br>
**Tip**: You can find this information in Endpoints section of your Cloud Object Storage instance dashbord.

In [ ]:
service_endpoint = 'https://s3.us.cloud-object-storage.appdomain.cloud'

You also need IBM Cloud authorization endpoint to be able to create COS resource object.

In [ ]:
auth_endpoint = 'https://iam.cloud.ibm.com/identity/token'

We create COS resource to be able to write data to Cloud Object Storage.

In [ ]:
cos = ibm_boto3.resource('s3',
                         ibm_api_key_id=cos_credentials['apikey'],
                         ibm_service_instance_id=cos_credentials['resource_instance_id'],
                         ibm_auth_endpoint=auth_endpoint,
                         config=Config(signature_version='oauth'),
                         endpoint_url=service_endpoint)

Now you will create bucket in COS and copy `training dataset` for model from **WA_FnUseC_TelcoCustomerChurn.csv**.

In [ ]:
from uuid import uuid4

bucket_uid = str(uuid4())

score_filename = "WA_FnUseC_TelcoCustomerChurn.csv"
buckets = ["churn-" + bucket_uid]

In [ ]:
for bucket in buckets:
    if not cos.Bucket(bucket) in cos.buckets.all():
        print('Creating bucket "{}"...'.format(bucket))
        try:
            cos.create_bucket(Bucket=bucket)
        except ibm_boto3.exceptions.ibm_botocore.client.ClientError as e:
            print('Error: {}.'.format(e.response['Error']['Message']))

In [ ]:
bucket_obj = cos.Bucket(buckets[0])

print('Uploading data {}...'.format(score_filename))
with open(filename, 'rb') as f:
    bucket_obj.upload_fileobj(f, score_filename)
print('{} is uploaded.'.format(score_filename))

## Create connections to a COS bucket

In [ ]:
datasource_type = client.connections.get_datasource_type_uid_by_name('bluemixcloudobjectstorage')

conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: "COS connection - spark",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: datasource_type,
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': buckets[0],
        'access_key': connection_access_key_id,
        'secret_key': connection_secret_access_key,
        'iam_url': auth_endpoint,
        'url': service_endpoint
    }
}

conn_details = client.connections.create(meta_props=conn_meta_props)

**Note:** The above connection can be initialized alternatively with api_key and resource_instance_id.
The above cell can be replaced with:

```
conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: f"Connection to Database - {db_name} ",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_uid_by_name(db_name),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucket_name,
        'api_key': cos_credentials['apikey'],
        'resource_instance_id': cos_credentials['resource_instance_id'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': 'https://s3.us.cloud-object-storage.appdomain.cloud'
    }
}

conn_details = client.connections.create(meta_props=conn_meta_props)
```

In [ ]:
connection_id = client.connections.get_uid(conn_details)

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [ ]:
training_data_references = [
                {
                    "id": "customer churn",
                    "type": "connection_asset",
                    "connection": {
                        "id": connection_id
                    },
                    "location": {
                        "bucket": buckets[0],
                        "file_name": score_filename,
                    }
                }
            ]

In [ ]:
saved_model = client.repository.store_model(
    model=model_lr, 
    meta_props={
        client.repository.ModelMetaNames.NAME:'Customer Churn model',
        client.repository.ModelMetaNames.TYPE: "mllib_3.3",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name('spark-mllib_3.3'),
        client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        client.repository.ModelMetaNames.LABEL_FIELD: "Churn",
    },  
    training_data=train_data, 
    pipeline=pipeline_lr)

Get saved model metadata from Watson Machine Learning.

In [ ]:
published_model_ID = client.repository.get_model_id(saved_model)

print("Model Id: " + str(published_model_ID))

**Model Id** can be used to retrive latest model version from Watson Machine Learning instance.

### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [ ]:
loaded_model = client.repository.load(published_model_ID)

In [ ]:
print(type(loaded_model))

As you can see the name is correct. You have already learned how save and load the model from Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally

In this section you will learn how to load data from batch scoring and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [ ]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [ ]:
predictions.show(5, truncate=False, vertical=True)

By tabulating a count, you can see the split between labels.

In [ ]:
predictions.select("prediction").groupBy("prediction").count().show(truncate=False)

<a id="scoring"></a>
## 6. Deploy model and score in Cloud

In this section you will learn how to create batch deployment and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](https://cloud.ibm.com/apidocs/machine-learning?code=python#deployments-create).

### 6.1 Prepare scoring data for batch job

#### Get data for prediction

First, download scoring data into notebook's filesystem

In [ ]:
import os
from wget import download

sample_dir = 'spark_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename = os.path.join(sample_dir, 'scoreInput.csv')
if not os.path.isfile(filename):
    filename = download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/customer_churn/scoreInput.csv", out=sample_dir)

### 6.2: Create batch deployment

Now you can create a batch scoring endpoint.
Execute the following sample code that uses the **published_model_ID** value to create the scoring endpoint for predictions.

#### Create batch deployment for published model

In [ ]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: "Customer Churn batch deployment",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 1
    }
}

deployment_details = client.deployments.create(published_model_ID, meta_props=meta_data)

Batch deployment has been created.

You can retrieve now your deployment ID.

In [ ]:
deployment_uid = client.deployments.get_uid(deployment_details)

You can also list all deployments in your space.

In [ ]:
client.deployments.list()

If you want to get additional information on your deployment, you can do it as below.

In [ ]:
client.deployments.get_details(deployment_uid)

#### Create and run Batch job

Tip: To install pandas execute `!pip install pandas`.

In [ ]:
import pandas as pd

score_input = pd.read_csv(filename).astype('object')

In [ ]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            "fields": score_input.columns.tolist(),
            "values": [score_input.loc[0].tolist()]
        }
    ]
}

In [ ]:
job = client.deployments.create_job(deployment_uid, meta_props=job_payload_ref)

Now, your job has been submitted to Spark runtime.

You can retrieve now your job ID.

In [ ]:
job_id = client.deployments.get_job_uid(job)

You can also list all jobs in your space.

In [ ]:
client.deployments.list_jobs()

If you want to get additional information on your job, you can do it as below.

In [ ]:
client.deployments.get_job_details(job_id)

#### Monitor job execution

Here you can check status of your batch scoring. When status of Spark job is `completed` the results will be written to scoring_output file in Object Storage.

In [ ]:
import time

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    print(f" Current state: {client.deployments.get_job_status(job_id).get('state')}")
    job_details_do = client.deployments.get_job_details(job_id)
    print(job_details_do)
else:
    print("Job hasn't completed successfully in 5 minutes.")

**Get scored data**

In [ ]:
import json

print(json.dumps(client.deployments.get_job_details(job_id), indent=1))

<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/data-science.html?context=cpdaas&audience=wdp)
 for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM

Copyright © 2020-2024 IBM. This notebook and its source code are released under the terms of the MIT License.